In [1]:
import altair as alt
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

### <font color="#747a7f">6.7.1</font> Filtragem Cruzada

Os exemplos de brushing & linking que nós analisamos até agora utilizam codificações condicionais, por exemplo para alterar valores de opacidade em resposta a uma seleção. Outra opção é usar uma seleção definida em uma visualização para *filtrar* o conteúdo de outra visualização.

Vamos construir uma coleção de histogramas para o dataset `flights`: o `delay` de chegada (o quão cedo ou tarde um voo chega, em minutos), `distance` (a distância percorrida em milhas) e `time` (hora do dia da partida). Nós usaremos o operador `repeat` para criar os histogramas e adicionar uma seleção de intervalo (`interval`) para o eixo `x`, com seleções filtradas pela interseção (brush).

Em particular, cada histograma consistirá em duas camadas: uma camada de fundo cinza e uma camada de primeiro plano azul, com a camada de primeiro plano filtrada pela interseção das seleções brush. O resultado é uma interação de *filtragem cruzada* entre os três gráficos!

*Arraste os intervalos de seleção nos gráficos abaixo. Ao selecionar voos com maiores ou menores atrasos na chegada, como as distribuições de distância e horário reagem?*


In [2]:
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
);

hist = alt.Chart().mark_bar().encode(
    alt.X(alt.repeat('row'), type='quantitative',
        bin=alt.Bin(maxbins=100, minstep=1), # até 100 intervalos
        axis=alt.Axis(format='d', titleAnchor='start') # formato inteiro, título alinhado à esquerda
    ),
    alt.Y('count():Q', title=None) # sem título no eixo y
)

alt.layer(
    hist.add_params(brush).encode(color=alt.value('lightgrey')),
    hist.transform_filter(brush)
).properties(
    width=900,
    height=100
).repeat(
    row=['delay', 'distance', 'time'],
    data=flights
).transform_calculate(
    delay='datum.delay < 180 ? datum.delay : 180', # fixa atrasos maiores que 3 horas em 180 minutos
    time='hours(datum.date) + minutes(datum.date) / 60' # horas fracionárias
).configure_view(
    stroke='transparent' # sem outline
)

alt.RepeatChart(...)

*Com a filtragem cruzada, você pode observar que voos atrasados têm mais chance de partir em horários mais tardios. Esse fenômeno é familiar para viajantes frequentes: um atraso pode se propagar ao longo do dia, afetando os voos subsequentes da mesma aeronave. Para aumentar as chances de uma chegada pontual, reserve um voo mais cedo!*

A combinação de múltiplas visualizações e seleções interativas pode possibilitar formas valiosas de análise multidimensional, transformando até mesmo histogramas básicos em poderosas ferramentas para fazer perguntas a um dataset!


## <font color="#747a7f">6.8</font> Sumário

Para mais informações sobre as opções de interação suportadas no Altair, por favor consulte a [documentação de seleção interativa do Altair](https://altair-viz.github.io/user_guide/interactions.html). Para detalhes sobre como personalizar manipuladores de eventos, por exemplo para compor várias técnicas de interação ou suportar entrada por toque em dispositivos móveis, consulte a [documentação de seleção do Vega-Lite](https://vega.github.io/vega-lite/docs/selection.html).

Interessado em aprender mais? - A abstração de *seleção* foi introduzida no artigo [Vega-Lite: A Grammar of Interactive Graphics](http://idl.cs.washington.edu/papers/vega-lite/), de Satyanarayan, Moritz, Wongsuphasawat e Heer. - O sistema PRIM-9 (para projeção, rotação, isolamento e mascaramento em até 9 dimensões) é uma das primeiras ferramentas de visualização interativas, desenvolvida no início dos anos 1970 por Fisherkeller, Tukey e Friedman. [Um vídeo demo retro ainda existe!](https://www.youtube.com/watch?v=B7XoW2qiFUA) - O conceito de brushing & linking foi consolidado por Becker, Cleveland e Wilks no artigo de 1987 [Dynamic Graphics for Data Analysis](https://scholar.google.com/scholar?cluster=14817303117298653693). - Para um resumo abrangente das técnicas de interação para visualização, consulte [Interactive Dynamics for Visual Analysis](https://queue.acm.org/detail.cfm?id=2146416) de Heer & Shneiderman. - Finalmente, para um tratado sobre o que torna a interação eficaz, leia o clássico artigo [Direct Manipulation Interfaces](https://scholar.google.com/scholar?cluster=15702972136892195211) de Hutchins, Hollan e Norman.


#### <font color="#747a7f">6.8.0.1</font> Apêndice: Sobre a Representação do Tempo

Anteriormente, nós observamos um pequeno pico no número de filmes em 1969 e 1970. De onde vem esse pico? E por que 1969 *ou* 1970? A resposta vem de uma combinação de dados ausentes e como o seu computador representa o tempo.

Internamente, datas e horários são representados em relação ao [epoch UNIX](https://en.wikipedia.org/wiki/Unix_time), onde o tempo "zero" corresponde à meia-noite de 1º de janeiro de 1970 no horário [UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time), que segue o [meridiano principal](https://en.wikipedia.org/wiki/Prime_meridian). Acontece que há alguns filmes com datas de lançamento ausentes (`null`). Esses valores `null` são interpretados como tempo `0`, e, portanto, mapeados para 1º de janeiro de 1970 no horário UTC. Se você mora nas Américas – e, portanto, em fusos horários "anteriores" – esse ponto exato no tempo corresponde a uma hora anterior em 31 de dezembro de 1969 no seu fuso horário local. Por outro lado, se você mora perto ou a leste do meridiano principal, a data no seu fuso horário local será 1º de janeiro de 1970.

A lição? Sempre seja cético com seus dados e tenha em mente que a forma como os dados são representados (seja como datas e horários, números de ponto flutuante, latitudes e longitudes, *etc.*) pode, por vezes, levar a artefatos que impactam a análise!
